In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl['dev']  # upgrade fastrl on colab

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

In [ ]:
# default_exp core

In [ ]:
# export
# Python native modules
import os,warnings
# Third party libs
from fastcore.all import *
from fastai.torch_core import *
from fastai.basics import *
import pandas as pd
import torch
import numpy as np
# Local modules

In [ ]:
# hide

# Core
> Core libs for fastrl

## D
> A better dictionary

In [ ]:
# export
def isnone(v): return v is None

In [ ]:
# export
def map_dict_ex(d,f,*args,gen=False,wise=None,**kwargs):
    "Like `map`, but for dicts and uses `bind`, and supports `str` and indexing"
    g = (bind(f,*args,**kwargs) if callable(f)
         else f.format if isinstance(f,str)
         else f.__getitem__)

    if wise is None:  return map(g,d.items())
    return ((k,g(v)) if wise=='value' else (g(k),v) for k,v in d.items())

Check that general mapping for dicts works nicely...

In [ ]:
test_dict={'a':1,'b':2,'c':3}
test_eq(dict(map_dict_ex(test_dict,lambda t:(t[0]+'_new',t[1]+1))),{'a_new':2,'b_new':3,'c_new':4})

Check that key and value wise mapping works correctly...

In [ ]:
test_eq(dict(map_dict_ex(test_dict,lambda k:k+'_new',wise='key')),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(map_dict_ex(test_dict,lambda v:v+1,wise='value')),{'a':2,'b':3,'c':4})

In [ ]:
# export
_error_msg='Found idxs: %s have values more than %s e.g.: %s'

class D(dict):
    "Improved version of `dict` with array handling abilities"
    def __init__(self,*args,mapping=False,**kwargs):
        self.mapping=mapping
        super().__init__(*args,**kwargs)
        
    def eq_k(self,o:'D',with_diff=False):
        eq=set(o.keys())==set(self.keys())
        if with_diff: return eq,set(o.keys()).symmetric_difference(set(self.keys()))
        return eq
    def _new(self,*args,**kwargs): return type(self)(*args,**kwargs)

    def map(self,f,*args,gen=False,**kwargs): 
        return (self._new,noop)[gen](map_dict_ex(self,f,*args,**kwargs),mapping=True)
    def mapk(self,f,*args,gen=False,wise='key',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)
    def mapv(self,f,*args,gen=False,wise='value',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)

In [ ]:
test_dict=D({'a':1,'b':2,'c':3})
test_eq(test_dict.map(lambda t:(t[0]+'_new',t[1]+1)),{'a_new':2,'b_new':3,'c_new':4})
test_eq(isinstance(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True),map),True)
test_eq(dict(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True)),{'a_new':2,'b_new':3,'c_new':4})

test_eq(test_dict.mapk(lambda k:k+'_new'),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(test_dict.mapk(lambda k:k+'_new',gen=True)),{'a_new':1,'b_new':2,'c_new':3})

test_eq(test_dict.mapv(lambda v:v+1,wise='value'),{'a':2,'b':3,'c':4})
test_eq(dict(test_dict.mapv(lambda v:v+1,gen=True,wise='value')),{'a':2,'b':3,'c':4})

`BD` is the primary data structure that `fastrl` uses. It allows for easily iterating and doing operations on steps attained from environments.

## BD 
> A batch wise dictionary that requires all values to be numpy,tensor, or None.

### Utilities

#### Stacking API
> We use the fastcore typedispatch decortors to select the currect method of stacking arrays.

In [ ]:
# export
@typedispatch
def stack(a:Tensor,b:Tensor): return torch.vstack((a,b))
@typedispatch
def stack(a:(np.array,np.ndarray),b:(np.array,np.ndarray)): return np.vstack((a,b))

#### Exceptions 

In [ ]:
# export 
class BDTypeError(Exception):
    def __init__(self,k,v):
        self.message=f'Key:{k} has value {type(v)} when it should have type Tensor, np.ndarray, or None'
        super().__init__(self.message)
    @classmethod
    def check(cls,d:dict,strict=False): 
        for k,v in d.items():
            if isinstance(k,int): raise TypeError(f'{k} cannot be an int since ints can be used to index.')
            types=(np.ndarray,NoneType)
            if not strict:types=(*types,list,L,Tensor)
            if not isinstance(v,(list,L,Tensor,np.ndarray,NoneType)): raise cls(k,v)
                
class BDBatchSizeError(Exception):
    def __init__(self,k,v,bs):
        self.message=f'Key:{k} has shape {v} when it should have shape[0]=={bs}'
        super().__init__(self.message)
    @classmethod
    def check(cls,d:'BD'): 
        for k,v in d.items():
            if v.shape[0]!=d.bd_batch_size: raise cls(k,v.shape,d.bd_batch_size)
                
class BDKeyMisMatchError(Exception):
    def __init__(self,different):
        self.message=f'Keys: {different} are not shared by the dictionaries'
        super().__init__(self.message)
    @classmethod
    def check(cls,d:'BD',other:'BD'): 
        eq,different=d.eq_k(other,with_diff=True)
        if not eq: raise cls(different)

#### Correction Functions

In [ ]:
# export
def list2tensor(l:(L,list)): return tensor(l) if not isinstance(l,np.ndarray) else l
@typedispatch
def batch_dim(o:Tensor,bs=1): return o.unsqueeze(0) if len(o.shape)<2 or o.shape[0]>bs else o
@typedispatch
def batch_dim(o:np.ndarray,bs=1): return np.expand_dims(o,0) if len(o.shape)<2 or o.shape[0]>bs else o

In [ ]:
# export
class BD(D):
    def __init__(self,*args,bd_batch_size=1,**kwargs):
        "Converts listy values to tensors, and keep np.ndarray and tensor values. Adds batch dims and validates them."
        super().__init__(*args,**kwargs)
        self.bd_batch_size=ifnone(bd_batch_size,self.bs)
        BDTypeError.check(self)
        if not self.mapping: 
            self.update(self.mapv(list2tensor).mapv(batch_dim,bs=self.bd_batch_size))
            self.mapping=False
        BDTypeError.check(self,strict=True)
        if not self.mapping: BDBatchSizeError.check(self)
    @property
    def bs(self):
        if len(self.values())==0: return 0
        return dict(self).popitem()[1].shape[0]
        
    def __getitem__(self,o):
        if isinstance(o,int) or is_listy(o): 
            bs=1 if isinstance(o,int) else None
            return BD({k:self[k][o] for k in self},bd_batch_size=bs)
        return super().__getitem__(o)
    
    def __add__(self,o):
        BDKeyMisMatchError.check(self,o)
        return BD({k:stack(self[k],o[k]) for k in self},
                  bd_batch_size=self.bd_batch_size+o.bd_batch_size)

We need to change any indexer that is passed. We don't know if the indexer is going to
be a numpy array, slice, tensor, or int.
All we know is 2 things:
- If it is an int, the batch dim will disappear
- If it is an indexer, then the batch dim will stay, but be smaller

In [ ]:
step_f=lambda: {'state':np.random.rand(4,),'next_state':torch.rand(4,),
                'action':np.random.randint(0,2),'reward':np.random.ranf(),
                'steps':np.random.randint(0,20),'episode_reward':np.random.randint(5,40),
                'env':np.random.randint(5,40),'done':np.random.randint(0,2)==0}
test_fail(lambda:BD(step_f()))
step_f=lambda: {'state':np.random.rand(4,),'next_state':torch.rand(4,),
                'action':[np.random.randint(0,2)],'reward':[np.random.ranf()],
                'steps':[np.random.randint(0,20)],'episode_reward':[np.random.randint(5,40)],
                'env':[np.random.randint(5,40)],'done':[np.random.randint(0,2)==0],
                'image':torch.rand(5,5,3)}

test_d=sum((BD(step_f()) for _ in range(19)),BD(step_f()))
test_eq(test_d.bd_batch_size,20)

In [ ]:
test_eq(type(test_d['state']),np.ndarray)
test_eq(type(test_d[5]),BD)
test_eq(test_d[5:8].bd_batch_size,3)

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted .data.block_old.ipynb.
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
No notebooks were modified
